<a href="https://colab.research.google.com/github/AnnSenina/python_hse_2024/blob/main/notebooks/3_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Операции с датафреймами

* Повторение: открытие df
* Работа со столбцами, выбор нескольких столбцов
* Типы данных: изменение типа данных (новые: category, dt)
* Перекодирование переменных. Понижение шкалы
  * for + if (медленный способ)
  * def + apply
  * lambda и списковые включение (list comprehensions)
  * из номинальных в бинарные переменные
* Соединение df

## 1. Открытие файлов

In [ ]:
import pandas as pd

df1 = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/BikeDataVar.csv')
# ИЛИ, если файл загружен в среду:
# df1 = pd.read_csv('BikeDataVar.csv')

# аргументы:
# sep=',' по умолчанию, можно изменить
# decimal='.' по умолчанию, можно изменить

df1.head(3)

In [ ]:
# в других средах, кроме колаб: может потребоваться
# !pip install openpyxl

df2 = pd.read_excel('BikeDataVar.xlsx')
df2.head(3)

# аргументы:
# sheet_name='Лист1', можно открыть конкретный лист
# skiprows=2, можно пропустить несколько строк

In [ ]:
data = pd.read_html('https://www.cbr.ru/currency_base/daily/')
data # возвращается список таблиц, т.к. на сайте их может быть несколько

In [ ]:
data[0]

### Практика

загрузите 9 таблицу со страницы https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BB%D0%B0%D0%B2_%D0%90%D0%B2%D1%81%D1%82%D1%80%D0%B8%D0%B8


Франц I, Фердинанд I, Франц Иосиф I и т.д.

In [ ]:
# ваше решение:



In [ ]:
# @title
data = pd.read_html('https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BB%D0%B0%D0%B2_%D0%90%D0%B2%D1%81%D1%82%D1%80%D0%B8%D0%B8')
data[8]

### Бонус

In [ ]:
# чтение json
myjson = """
[
{
    "firstName": "Анна",
    "lastName": "Сенина"
},
{
    "firstName": "Никита",
    "lastName": "Маткин"
}
]
"""

df3 = pd.read_json(myjson)
df3

In [ ]:
# загрузим метаданные DraCor

import requests
import pandas as pd
json_url = 'https://dracor.org/api/corpora/rus/metadata'
response = requests.get(json_url)

df4 = pd.read_json(response.text)
df4.head(3)

## Работа со столбцами, выбор нескольких столбцов

In [ ]:
df1 = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/BikeDataVar.csv')
df1.head(3)

In [ ]:
# мы можем собрать небольшой датафрейм, выбрав только нужные столбцы
df1.columns

In [ ]:
mini = df1[['Date', 'Hour', 'Temperature', 'Humidity', 'Seasons', 'Rental Count']]
mini

# обычно я предпочитаю не удалять столбцы, а работать с выбранными, если нужна компактная таблица

In [ ]:
df1[df1.columns[:5]] # можно задать срез нужных столбцов

In [ ]:
df1[['Rental Count', 'Date']] # или поменять их порядок

In [ ]:
# столбцы можно складывать, вычитать, умножать, делить и мн.другое - подобно массивам numpy
df1['Rainfall'] + df1['Snowfall']

In [ ]:
df1['Snow_or_rain'] = df1['Rainfall'] + df1['Snowfall']
df1.head()

## Типы данных

На основе [этой](https://dfedorov.spb.ru/pandas/%D0%9E%D0%B1%D0%B7%D0%BE%D1%80%20%D1%82%D0%B8%D0%BF%D0%BE%D0%B2%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20pandas.html) тетрадки

<table>
<thead><tr>
<th>Pandas</th>
<th>Python</th>
<th>NumPy</th>
<th>Использование</th>
</tr>
</thead>
<tbody>
<tr>
<td>object</td>
<td>str или разные</td>
<td>string<em>, unicode</em>, смешанные типы</td>
<td>Текстовые или смешанные числовые и нечисловые значения</td>
</tr>
<tr>
<td>int64</td>
<td>int</td>
<td>int_, int8, int16, int32, int64, uint8, uint16, uint32, uint64</td>
<td>Целые числа</td>
</tr>
<tr>
<td>float64</td>
<td>float</td>
<td>float_, float16, float32, float64</td>
<td>Числа с плавающей точкой</td>
</tr>
<tr>
<td>bool</td>
<td>bool</td>
<td>bool_</td>
<td>Значения True/False</td>
</tr>
<tr>
<td>datetime64[ns]</td>
<td>datetime</td>
<td>datetime64[ns]</td>
<td>Значения даты и времени</td>
</tr>
<tr>
<td>category</td>
<td>NA</td>
<td>NA</td>
<td>Ограниченный список текстовых значений</td>
</tr>
</tbody>
</table>


### Категориальные данные

Их можно оставить object (строка), но лучше преобразовать в тип данных category

Подробнее о том, зачем это делать, [здесь](https://dfedorov.spb.ru/pandas/%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%82%D0%B8%D0%BF%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8%20%D0%B2%20pandas.html#:~:text=%D0%A2%D0%B8%D0%BF%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8%20(%20category%20data,%D0%B8%20%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5%20%D1%8D%D1%84%D1%84%D0%B5%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%20%D0%B8%D1%85%20%D1%85%D1%80%D0%B0%D0%BD%D0%B8%D1%82%D1%8C.)

In [ ]:
df1['Temperature Category'].astype('category')

In [67]:
df1['Temperature Category'] = df1['Temperature Category'].astype('category') # перезапишем столбец

In [ ]:
df1.info()

### Даты

In [ ]:
df1['Date'] = df1['Date'].astype('datetime64[ns]')

In [ ]:
df1.info()

In [ ]:
# примеры с датами в разных форматах
pd.Series(['05/23/2005']).astype('datetime64[ns]')

In [ ]:
pd.to_datetime(pd.Series(['05/23/2005']), format="%m/%d/%Y")

Во многом, такие преобразования нужны, поскольку pandas умеет по дате получить день, месяц, год, день недели, номер недели, квартал и мн.другое

In [ ]:
df1['Date'] = df1['Date'].astype('datetime64[ns]')
df1['Date'].dt.day
# dt.month
# dt.year
# dt.isocalendar().week
# dt.days_in_month
# dt.day_of_year
# dt.day_of_week
# dt.day_name()
# df.quarter

### Пратика

Добавьте в датафрейм столбец с именем day_of_week, в котром будет **название** для недели

In [ ]:
# ваш код



In [ ]:
# @title

df1['day_of_week'] = df1['Date'].dt.day_name()
df1

### Числа

Если ```.astype('int64')``` не сработал (например, встретил нечисловое значение), можно использовать более гибкий вариант

In [ ]:
df1['Rental Count'].astype('int64')

In [ ]:
# у нас с этим столбцом все в порядке, но на будущее:

pd.to_numeric(df1['Rental Count'], errors='coerce')
# errors - способ обработки ошибок, их несколько
# errors='coerce' превращает проблемные значения в NaN

In [ ]:
# тоже на будущее:
# оставим в df1 только числовые столбцы

df1.select_dtypes(include='number')

# select_dtypes(include='int64')
# select_dtypes(include=['number', 'bool']) и т.д.

## Перекодирование переменных

In [ ]:
# Давайте перекодируем переменную Temperature Category
# Freezing, Chilly - 0, Nice, Hot - 1

In [ ]:
# 1 способ

l = []

for i in df1['Temperature Category']:
  if i == 'Freezing' or i == 'Chilly':
    l.append(0)
  else:
    l.append(1)

df1['coded'] = l

df1.head(3)

In [ ]:
# 2 способ - лучше - нативный для pandas

def f(n):
  if i == 'Freezing' or i == 'Chilly':
    return 0
  else:
    return 1

df1['coded'] = df1['Temperature Category'].apply(f)

df1.head(3)

In [ ]:
# 3 способ (разновидность предыдущего)
# списковые включение (list comprehensions)

df1['Temperature Category'].apply(lambda x: 0 if x == 'Freezing' or x == 'Chilly' else 1)

In [ ]:
# усложним:
df1['Temperature Category'].apply(lambda x: 1 if x == 'Freezing' else
                                           (2 if x == 'Chilly' else
                                           (3 if x == 'Nice' else 4)))

### Бонус

Другие примеры списковых включений:

In [ ]:
l = []

for i in range(10):
  if i % 2 == 0:
    l.append(i)

print(l)

In [ ]:
# аналогично
print([i for i in range(10) if i % 2 == 0])

In [ ]:
l = []
for i in range(10):
  if i % 2 == 0:
    l.append(True)
  else:
    l.append(False)

print(l)

In [ ]:
# аналогично
print([True if i % 2 == 0 else False for i in range(10)])

Перекодирование номинальных переменных в бинарные

In [ ]:
# перекодируем сезоны, уопрядочить их от зимы к осени - не очень корректно
# в этом случае pandas будет считать, что осень в 4 раза больше зимы
# можно добавить 4 новых столбика: зима / нет, весна / нет и т.д.
# писать 4 функции для 4 новых столбцов неудобно

pd.get_dummies(df1['Seasons'])

In [ ]:
pd.get_dummies(df1['Seasons']).astype('int64')

## Соединение таблиц

In [ ]:
# Простой пример с прошлой пары:
data = [
    ["Maria", "London", 37],
    ["Lorenzo", "Milan", 28],
    ["Oleg", "Canberra", 31],
    ["Hans", "Calgary", 80],
    ["Mark", "Milan", 55],
    ["Alex", "Krakow", 35],
    ["Julia", "Murmansk", 43]
    ]

df = pd.DataFrame(data, columns=['name', 'city', 'age'])
df

In [ ]:
l = [
    ['Alice', 'NY', 36],
    ['Greta', 'Berlin', 30]
    ]

new = pd.DataFrame(l, columns=['name', 'city', 'age'])
new

In [ ]:
pd.concat([df, new])
# pd.concat([df, new], axis=0)
# axis = 0 по умолчанию добалвяет новые строки
# для добавления столбцов можно использовать axis=1

In [ ]:
l = [
    ['Nick', 'Barcelona', 45, 'Chef'],
    ['Sybil', 'Warsaw', 30, 'Artist']
    ]

new2 = pd.DataFrame(l, columns=['name', 'city', 'age', 'work'])
new2

In [ ]:
pd.concat([df, new, new2]) # соединяет строки
# доб. агрумент axis=1 и сможем соединять столбцы

О слиянии в стиле SQL [здесь](https://miptstats.github.io/courses/python/10_pandas2.html)

### Бонус

In [ ]:
import geopandas as gpd # геопандас - для работы с координатами
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
happiness = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Other/main/WHR2023.csv')
happiness.head()

In [ ]:
# world['name'] и happiness['Country name'] совпадают, соединим
data_world = pd.merge(world, happiness, left_on='name', right_on='Country name') # соединили 2 датасета
data_world

In [ ]:
data_world.plot(column ='Ladder score', legend=True);

## Текстовые данные

Текст часто бывает удобно представить в виде таблицы

In [179]:
with open('classic.txt') as f:
  text = f.read()

In [ ]:
# словарь абсолютных частот слов
from collections import Counter
Counter(text.lower().split())

In [ ]:
words = pd.DataFrame.from_dict(Counter(text.lower().split()), orient='index', columns=['freq'])
words

In [ ]:
# Или:
with open('classic.txt') as f:
  text_list = f.readlines()

sent = pd.DataFrame(text_list, columns=['sent'])
sent # позже сюда можно добавить результаты анализа тональности и др.

# Задачи для тренировки

Вы работаете с датасетом от Гарвардского университета, в котором хранится информация о государствах - бывших колониях.

Перекодируйте переменные **decolonization, foreign trade, plantations,	gold/silver**

Скачать данные можно здесь: https://raw.githubusercontent.com/AnnSenina/Python_CL_2023/main/data/Colonial.csv

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Python_CL_2023/main/data/Colonial.csv', sep=';')
df

Возможные решения:

In [ ]:
# @title
def f(n):
  if n == 'short':
    return 0
  else:
    return 1

df['coded_decolonization'] = df['decolonization'].apply(f)

In [ ]:
# @title

new = pd.get_dummies(df['decolonization']).astype('int64')
pd.concat([df, new], axis=1)